## Prérequis :
- Avoir installé rust
- Avoir installé maturin (pip install maturin)
- Avoir crée et activé un environnement virtuel python (`python -m venv venv && source venv/bin/activate`)

## Compilation (test)
```bash
maturin develop
```
> A executer à chaque modification du code rust dans l'environnement virtuel python

## Compilation (release)
```bash
maturin build
pip install target/wheel/*.whl
```
> A executer pour compiler le code rust en release

# Usage
Le module installé a en python un nom correspondant a la propriété `lib.name` du fichier `Cargo.toml` 
ex :
```
[package]
name = "ProjetAnnuel"
version = "0.1.0"
edition = "2021"

[lib]
name = "projetannuel"
crate-type = ["cdylib"]

[dependencies]
pyo3 = {version= "0.24.0", features = ["extension-module"] }
```

Le module python s'appellera `projetannuel` et sera importable en python avec `import projetannuel`

In [26]:
import projetannuel


In [27]:
import pandas as pd

In [28]:
df = pd.read_csv("/home/ed0uard/Téléchargements/iris.csv", sep=';')

# shuffle dataframe with seed 0
df = df.sample(frac=1, random_state=0)

df

,longueur de pétale (cm),Largeur de pétale,Longueur de sépale,Largeur de sépale,ID,Espèce
114,6.3,1.8,7.3,2.9,c3df88ce-1f25-478d-9c11-80518230bad7,Iris-virginica
62,6.4,2.0,7.9,3.8,646595be-1cac-4b25-bcc6-bc5305fd201b,Iris-virginica
33,6.6,2.1,7.6,3.0,397ef579-78e8-48b4-aa5a-eadec82e651e,Iris-virginica
107,6.9,2.3,7.7,2.6,b6eb4add-bf92-4cd3-b006-97751f42ecf7,Iris-virginica
7,3.9,1.1,5.6,2.5,0ce4515f-8e31-4bf6-a8ef-13a1f702b708,Iris-versicolor
...,...,...,...,...,...,...
9,5.6,1.8,6.3,2.9,0dd2c701-49ca-489b-9abe-8ae3aef7b831,Iris-virginica
103,1.3,0.2,4.4,3.2,ae343a80-4c0b-4c8f-9b3f-afd689f67955,Iris-setosa
67,4.3,1.3,6.2,2.9,75df5623-4271-4760-9fad-99a8a62fb4fc,Iris-versicolor
117,4.9,1.5,6.3,2.5,c660988b-a4b4-4261-9f14-d8eab78b2a38,Iris-versicolor


In [29]:
import numpy as np

def split_dataframe(df, test_ratio=0.2, seed=42):
    np.random.seed(seed)
    shuffled_indices = np.random.permutation(len(df))
    test_set_size = int(len(df) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return df.iloc[train_indices], df.iloc[test_indices]

train_df, test_df = split_dataframe(df, test_ratio=0.2)

In [30]:
Y_train = [x for x in train_df["Espèce"]]
Y_test = [x for x in test_df["Espèce"]]
Y_train = projetannuel.StringLabels(Y_train)
Y_train, len(Y_test)

(<LabelsEnum_Str at 0x749e38b45e90>, 30)

In [31]:
df.columns

Index(['longueur de pétale (cm)', 'Largeur de pétale', 'Longueur de sépale',
       'Largeur de sépale', 'ID', 'Espèce'],
      dtype='object')

In [32]:
X = train_df[['longueur de pétale (cm)', 'Largeur de pétale', 'Longueur de sépale','Largeur de sépale']].values

In [33]:
model = projetannuel.LinearModel(
    X
    ,
    Y_train
)

model

In [34]:
model.train_classification(5_000_000,0.0001)

In [35]:
Y_pred = []

for v in test_df[['longueur de pétale (cm)', 'Largeur de pétale', 'Longueur de sépale','Largeur de sépale']].values :
    pred = model.predict(v)
    Y_pred.append(pred)
    
Y_pred

['Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-virginica',
 'Iris-versicolor',
 'Iris-versicolor']

In [36]:
val = [
    k==v for k,v in zip(Y_test,Y_pred)
]
sum(val) / len(val) * 100 

63.33333333333333